In [1]:
import cv2
import os
import numpy as np
import time
import dlib
import face_recognition
from imutils import face_utils
from scipy.spatial import distance as dist
from threading import Thread
import playsound
from pygame import mixer

pygame 1.9.6
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
detector = dlib.get_frontal_face_detector()#face detection model(hog+svm)
#cnn_face=dlib.cnn_face_detection_model_v1("weight")#if gpu enabled
predictor=dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")#Landmark file

In [3]:
EYE_AR_THRESH = 0.25#EAR threshold
EYE_AR_CONSEC_FRAMES = 8#For how many frame
COUNTER = 0
ALARM_ON = False

In [4]:
def eye_aspect_ratio(eye):#Vertical and horizontal distance of the eye
    A=dist.euclidean(eye[1],eye[5])
    B=dist.euclidean(eye[2],eye[4])
    C=dist.euclidean(eye[0],eye[3])
    
    ear=(A+B)/(2*C)
    return ear
                     

In [5]:
def sound_alarm(path):
    playsound.playsound(path)
mixer.init()
sound = mixer.Sound('alarm1.wav')

In [6]:
(lStart, lEnd) = face_utils.FACIAL_LANDMARKS_IDXS["left_eye"]#Left and right eye co-ordinates/indexes(36-48)
(rStart, rEnd) = face_utils.FACIAL_LANDMARKS_IDXS["right_eye"]

In [7]:
video=cv2.VideoCapture(0)


while True:
    check,frame=video.read()
    gray=cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
    #cv2.imshow("Capturing",gray)
    faces=detector(gray)#face detected
    #print(faces)
    
    for face in faces:#co-oordinates of the face detected
        x1=face.left()
        y1=face.top()
        x2=face.right()
        y2=face.bottom()
        cv2.rectangle(frame,(x1,y1),(x2,y2),(0,255,0),3)
        landmark=predictor(gray,face)#Landmark prediction on the detected image
        shape = face_utils.shape_to_np(landmark)#creating a numpy array
        lefteye=shape[lStart:lEnd]#slicing the left and right eye co-ordinates from the detected face
        righteye=shape[rStart:rEnd]
        leftEAR = eye_aspect_ratio(lefteye)#EAR of left and right eye
        rightEAR = eye_aspect_ratio(righteye)
        ear=(leftEAR+rightEAR)/2
        
        if ear<EYE_AR_THRESH:#if avg EAR less than the threshold
            COUNTER+=1
            #print(COUNTER)
            if COUNTER>=EYE_AR_CONSEC_FRAMES:#if avg EAR less than the threshold for specific amount of frames
                cv2.putText(frame, "DROWSINESS ALERT!", (10, 30),cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
                sound.play()
                
        else:
            COUNTER=0
            ALARM_ON=False
        cv2.putText(frame, "EAR: {:.2f}".format(ear), (300, 30),cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
             
        #print(landmark)
        for i in range(36,48):
            x=landmark.part(i).x
            y=landmark.part(i).y
            cv2.circle(frame,(x,y),2,(255,0,0),-1)
            
        
    cv2.imshow("Detection",frame)
    key=cv2.waitKey(1)
    if key==ord('q'):
        break

video.release()
cv2.destroyAllWindows()